In [1]:
pip install --force-reinstall agent_system/dist/mesa_restaurant_agents-0.0.1-py3-none-any.whl

In [2]:
import os
import sys
import pandas as pd
import plotly.graph_objects as go
import itertools

import mesa
from mesa_restaurant_agents.model.restaurant_model import RestaurantModel
from mesa_restaurant_agents.visualization import *
from mesa_restaurant_agents.visualization import GridAnimator

# Parameter Values

In [3]:
grid_side_lengths = [5, 10, 15, 20, 30]
waiter_counts = [1, 2, 5, 10, 15]
days = [1, 2, 5, 15, 30]

# Functions

In [4]:
def run_model(run):
    steps = run['days']*300
    params = {"n_waiters":run['waiter_init_count'], "grid_width":run['grid_side_length'], "grid_height": run['grid_side_length']}

    results = mesa.batch_run(
        RestaurantModel,
        parameters=params,
        iterations=1,
        max_steps=steps,
        number_processes=1,
        data_collection_period=1,
        display_progress=True,
    )

    # df = pd.DataFrame(results)
    # numeric_cols = ['Step', 'Customer_Count', 'Average_Wait_Time', 'Average_Customer_Satisfaction', 'Revenue']
    # df[numeric_cols] = df[numeric_cols].fillna(0).astype(float)
    
    # non_empty_cols = [col for col in df.columns if df[col].notna().any()]
    # df[non_empty_cols].head()

    return results


def suppress_print(func):
    def wrapper(*args, **kwargs):
        original_stdout = sys.stdout
        sys.stdout = open(os.devnull, 'w')
        result = func(*args, **kwargs)
        sys.stdout.close()
        sys.stdout = original_stdout
        return result
    return wrapper


run_model_print_suppressed = suppress_print(run_model)


def get_revenue_list(results):
    df = pd.DataFrame(results)
    daily_stats = df['Daily_Stats'].tolist()
    unique_daily_stats = []
    seen = set()
    for sublist in daily_stats:
        for item in sublist:
            item_tuple = tuple(item.items())
            if item_tuple not in seen:
                seen.add(item_tuple)
                unique_daily_stats.append(item)
    unique_daily_stats = [item for item in unique_daily_stats if item]
    daily_revenue = [stats['revenue'] for stats in unique_daily_stats]
    return daily_revenue


def print_analytics(daily_revenue, run):
    fig = go.Figure(data=[go.Bar(x=list(range(len(daily_revenue))), y=daily_revenue)])

    fig.update_layout(
        title=f'Daily Revenue for GridSize {run['grid_side_length']}, InitWaiters {run['waiter_init_count']} and Days {run['days']}',
        xaxis_title='Day',
        yaxis_title='Revenue (Money)'
    )
    fig.show()

def visualize_overview(overview):
    x_values = [f"Grid: {item['run']['grid_side_length']}, Waiters: {item['run']['waiter_init_count']}, Days: {item['run']['days']}" for item in overview]
    y_values = [item['mean_revenue'] for item in overview]

    fig = go.Figure(data=[go.Bar(x=x_values, y=y_values)])

    fig.update_layout(
        title="Mean Revenue by Run",
        xaxis_title="Run",
        yaxis_title="Mean Revenue",
        xaxis_tickangle=-45
    )

    fig.show()

# Visualization Example

In [5]:
results = run_model({"grid_side_length": 10, "waiter_init_count": 10, "days": 1})

  0%|          | 0/1 [00:00<?, ?it/s]

Day 1, Hour 12:00:
Customers paid: 0
Customers left without paying: 0
Current Revenue: $0.00

Day 1, Hour 13:00:
Customers paid: 3
Customers left without paying: 0
Current Revenue: $66.30

Day 1, Hour 14:00:
Customers paid: 15
Customers left without paying: 2
Current Revenue: $331.50

Starting shift 2: Afternoon
Shift 2: 2 waiters needed, 10 currently active
Removed 8 excess waiters
Day 1, Hour 15:00:
Customers paid: 29
Customers left without paying: 34
Current Revenue: $698.70

Day 1, Hour 16:00:
Customers paid: 31
Customers left without paying: 79
Current Revenue: $749.70

Day 1, Hour 17:00:
Customers paid: 31
Customers left without paying: 114
Current Revenue: $749.70

Day 1, Hour 18:00:
Customers paid: 31
Customers left without paying: 118
Current Revenue: $749.70

Starting shift 3: Evening
Shift 3: 2 waiters needed, 2 currently active
No change in waiters for shift 3
Day 1, Hour 19:00:
Customers paid: 31
Customers left without paying: 139
Current Revenue: $749.70

Day 1, Hour 20:0

In [6]:
animator = GridAnimator(results)

ani = animator.animate_first_run()
ani.save('animated_heatmap.gif', writer='pillow', fps=2, dpi=80)

In [7]:
grouped_data = display_mean_step_results(results)

In [8]:
customer_infos_df = display_first_run_step_results_customer(results)
waiter_infos_df = display_first_run_step_results_waiter(results)

None


None


None


None


# Parameter Comparison

In [5]:
runs = [{"grid_side_length": gsl, "waiter_init_count": wc, "days": d} for gsl, wc, d in itertools.product(grid_side_lengths, waiter_counts, days)]
results_list = []
overview = []
best_run_counter = 0
best_run_revenue = 0
counter = 0
for run in runs:
    results = run_model_print_suppressed(run)
    results_list.append(results)
    daily_revenue = get_revenue_list(results)
    mean_revenue = sum(daily_revenue)/run['days']
    overview.append({"run": run, "mean_revenue": mean_revenue})
    if mean_revenue > best_run_revenue:
        best_run_counter = counter
        best_run_revenue = mean_revenue
    counter += 1

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

...

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

In [6]:
visualize_overview(overview)
best_results = results_list[best_run_counter]